Example of use of the Isochrones analysis.

We'll need to setup credentials that allow us to import a table and modify it; for convenience we'll used stored credentials, which you can save like this:

```python
from cartoframes.auth import Credentials
Credentials(username='...', api_key='...').save()
```

In [1]:
from cartoframes.auth import set_default_credentials, Credentials
credentials = Credentials.from_file()
set_default_credentials(credentials)

We'll use a dataset with some locations:

In [11]:
from cartoframes.data import Dataset
dataset = Dataset("SELECT * FROM carto_places WHERE country='Spain'")

First we need an instace of the Routing service class (note that the default credentials will be used to access the service):

In [12]:
from cartoframes.data.services import Routing
routing = Routing()

We'll compute three isochrones for each point, corresponding to 20, 30 and 60 minute drives (given in seconds). But before doing any actual processing we'll perform a *dry run* to learn how many credits it would cost us to the job:

In [13]:
estimate = routing.isochrones(dataset, [1200,2400,3600], mode='car', dry_run=True).metadata
estimate

{'required_quota': 6}

Since we intend to visualize the result in a map, we'll add the `exclusive` parameter to generate non-overlapping areas corresponding to _exclusive_ interval ranges for 0 to 1200, 1200 to 2400 and 2400 to 3600 seconds. Otherwise areas would overlap and smaller areas may be obscured by the larger ones in the map.

Note that, in order to the estimate quota usage from the dry run to be applicable, we should not change the dataset or ranges argumente in the final call.

In [14]:
isochrones = routing.isochrones(dataset, [1200,2400,3600], mode='car', exclusive=True).data
isochrones

Now let's see how it actually looks in a map:

In [15]:
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import color_bins_layer
Map([color_bins_layer(isochrones, 'data_range',method='equal',bins=3), Layer(dataset)])